# Prediction generation on M0 .. M5 models using nnsight

This notebook:
- runs with models M0 to M5 and command sets CS1 to CS3
- uses the generate_csN functions to generate prompts
- uses nnsight call "model.generate(inputs['input_ids'], max_new_tokens=max_new_tokens, ...)" to generate a full answer
- scores the prediction accuracy uasing evaluate_csN_prediction and also "exact match"

This notebook was:
- Developed on Google Colab using an A100 with 40GB GPU and 80GB system RAM.
- Runs with GPT2/TinyStories/Qwen/Llama/Granite/SmolLM with base/CS1/CS2/CS3.
- Requires a GITHUB_TOKEN secret to access Martian TinySQL code repository.
- Requires a HF_TOKEN secret to access Martian HuggingFace repository.


# Import libraries
Imports standard libraries. Do not read.

In [ ]:
# https://nnsight.net/
!pip install -U nnsight

In [ ]:
from IPython.display import clear_output
import einops
import torch
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "colab"

import nnsight
from nnsight import LanguageModel, util
from nnsight.tracing.Proxy import Proxy

In [ ]:
from getpass import getpass
from google.colab import userdata
import gc
import weakref

In [ ]:
github_token = userdata.get("GITHUB_TOKEN")

# Install the private repository using the token
!pip install --upgrade git+https://{github_token}@github.com/withmartian/TinySQL.git

import TinySQL as qts

# Select model, command set and feature to investigate


In [ ]:
model_num = 3                 # 0=GPT2, 1=TinyStories, 2=Qwen, 3=Llama, 4=Granite, 5=SmolLM
cs_num = 1                    # 0=BaseModel, 1=CS1, 2=CS2 or 3=CS3
max_new_tokens = 100          # Max number of tokens to generate

# Run m0 : nnsight tutorial using GPT2
Based on https://nnsight.net/notebooks/tutorials/activation_patching/



In [ ]:
if model_num == 0:
    model0 = LanguageModel("openai-community/gpt2", device_map="auto")
    clear_output()
    print(model0)

In [ ]:
if model_num == 0: # GPT2
    the_prompt = "After John and Mary went to the store, Mary gave a bottle of milk to"
    print("Model input: ", the_prompt)
    inputs = model0.tokenizer(the_prompt, return_tensors="pt", padding=True)

    with model0.generate(inputs['input_ids'], max_new_tokens=25, pad_token_id=model0.tokenizer.eos_token_id) as tracer:
        final_output = model0.generator.output.save()

    final_output = final_output.detach().cpu().numpy()

    decoded_output = model0.tokenizer.decode(final_output[0], skip_special_tokens=True)
    print("Model output:", decoded_output)

# Run m1, m2 and m3 models

In [ ]:
if model_num > 0:
    model = qts.load_tinysql_model(model_num, cs_num, auth_token=userdata.get("HF_TOKEN"))
    clear_output()
    print(model)

In [ ]:
if model_num > 0:
    # Generate a batch of prompts
    batch_size = 50

    examples = qts.generate_csn(batch_size=batch_size, csn=max(1,cs_num))

In [ ]:
if model_num > 0:

    score1_sum = 0
    score2_sum = 0
    for idx in range(batch_size):
        example = examples[idx]

        the_prompt = example.get_alpaca_prompt()
        #print("Run:", idx, "Model input:", the_prompt)

        inputs = model.tokenizer(the_prompt, return_tensors="pt", padding=True)
        with model.generate(inputs['input_ids'], max_new_tokens=max_new_tokens, pad_token_id=model.tokenizer.eos_token_id) as tracer:
            final_output = model.generator.output.save()

        final_output = final_output.detach().cpu().numpy()
        decoded_output = model.tokenizer.decode(final_output[0], skip_special_tokens=True)

        assert decoded_output.startswith(the_prompt)
        model_added = decoded_output[len(the_prompt):]
        #print("Run:", idx, "Model added:", model_added)

        score1 = 0
        if cs_num == 0 or cs_num == 1:
            score1 = qts.evaluate_cs1_prediction(example, model_added)
        elif cs_num == 2:
            score1 = qts.evaluate_cs2_prediction(example, model_added)
        elif cs_num == 3:
            score1 = qts.evaluate_cs3_prediction(example, model_added)
        score2 = 1 if model_added == example.sql_statement else 0

        if score1 < 1 or score2 < 0:
          print("Run:", idx, "Score1:", score1, "Score2:", score2)

        score1_sum += score1
        score2_sum += score2

    print()
    print("Averages: Score1:", score1_sum/batch_size, "Score2:", score2_sum/batch_size)